In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [2]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.reset_index(inplace=True)
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,5,5263,555,1,1,0,1
1,2,2015-07-31,5,6064,625,1,1,0,1
2,3,2015-07-31,5,8314,821,1,1,0,1
3,4,2015-07-31,5,13995,1498,1,1,0,1
4,5,2015-07-31,5,4822,559,1,1,0,1


In [3]:
test_df = DataFrame.from_csv("test.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
test_df.reset_index(inplace=True)
test_df.head()

,Store,Date,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-09-17,1,4,1,1,0,0
1,3,2015-09-17,2,4,1,1,0,0
2,7,2015-09-17,3,4,1,1,0,0
3,8,2015-09-17,4,4,1,1,0,0
4,9,2015-09-17,5,4,1,1,0,0


# "Train" the model: Calculate mean per group over full training set

In [4]:
train_df = train_df[train_df['Sales'] > 0]
train_df['Open'] = train_df['Open'].fillna(1)


In [5]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means

means = calculate_means(train_df)
train_df = pandas.merge(train_df, means, on = ['Store','DayOfWeek','Promo'], how='left', suffixes=["", "_predicted"])
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())


In [24]:
means.head()

,Store,DayOfWeek,Promo,Sales
0,1,1,0,4310.446429
1,1,1,1,5852.708333
2,1,2,0,3897.145161
3,1,2,1,5364.597222
4,1,3,0,3869.590164


### Score the model

In [6]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [7]:
score_df = train_df[['Sales', 'Sales_predicted']].copy()
score_df = score_df[score_df['Sales'] != 0]
score_df['PercentError'] =  (score_df['Sales'] - score_df['Sales_predicted']) / score_df['Sales']
score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
math.sqrt(score_df['PercentErrorSquared'].mean())

0.22978844842293555

## Featurize and save

In [8]:
train_df.set_index(['Date', 'Store'], inplace=True)
test_df.set_index(['Date', 'Store'], inplace=True)


In [9]:
train_df['Sales-prediction_mean_error'] = (train_df['Sales_predicted'] - train_df['Sales']) / train_df['Sales']
train_df['Sales-prediction_mean_error'] = train_df['Sales-prediction_mean_error'].fillna(0) # when sales is 0, sales_predicted is nan.

In [13]:
train_df[['Sales_predicted', 'Sales-prediction_mean_error']].to_csv("train-features-predicted_mean.csv")

In [18]:
test_df.reset_index(inplace=True)

In [19]:

test_df = pandas.merge(test_df, means, on = ['Store','DayOfWeek','Promo'], how='left')

In [22]:
test_df

,Date,Store,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Sales_predicted
0,2015-09-17,1,1,4,1,1,0,0,4961.029412
1,2015-09-17,3,2,4,1,1,0,0,7994.426471
2,2015-09-17,7,3,4,1,1,0,0,9406.309859
3,2015-09-17,8,4,4,1,1,0,0,7194.929577
4,2015-09-17,9,5,4,1,1,0,0,6777.617647
5,2015-09-17,10,6,4,1,1,0,0,5844.126761
6,2015-09-17,11,7,4,1,1,0,0,8613.845070
7,2015-09-17,12,8,4,1,1,0,0,8490.760563
8,2015-09-17,13,9,4,1,1,0,0,5578.750000
9,2015-09-17,14,10,4,1,1,0,0,6157.676471


In [20]:
test_df['Sales_predicted'] = test_df['Sales']
del test_df['Sales']

In [21]:
test_df['Sales-prediction_mean_error'] = (test_df['Sales_predicted'] - test_df['Sales']) / test_df['Sales']
test_df['Sales-prediction_mean_error'] = test_df['Sales-prediction_mean_error'].fillna(0) # when sales is 0, sales_predicted is nan.

KeyError: 'Sales'

In [ ]:
test_df[['Sales_predicted']].to_csv("test-features-predicted_mean.csv")